In [123]:
import pandas as pd
import numpy as np

filepath="D:\\GitHub\\eEcoLiDAR\\Annotation\\Data\\"

speciestable=pd.read_csv(filepath+'speciestable.csv',sep=';',encoding='latin-1')
plottable=pd.read_csv(filepath+'plottable.csv',sep=';',encoding='latin-1')
observationtable=pd.read_csv(filepath+'observationtable.csv',sep=';',encoding='latin-1')

Extract table for Daniel for tree species selection (into excel)
- all data

In [125]:
speciestable.to_excel(filepath+'SpeciesTableOriginal.xlsx',encoding='latin-1',index=False)

Filter the polygons between 2007 and 2015 -- AHN2 was measured between 2008-2012

In [126]:
observationtable_up2007=observationtable[(observationtable["year"]>2007)]

Join: observation table to plottable

In [127]:
joined_obsplot=pd.merge(observationtable_up2007, plottable, on='polygonID', how='left')
joined_obsplot = joined_obsplot[list(joined_obsplot.columns[~joined_obsplot.columns.duplicated()])]
joined_obsplot_forest=joined_obsplot[(joined_obsplot["habitat"]=="Forest")]

Extract table for Daniel for tree species selection (into excel)
- just for forest

In [141]:
SpeciesTable_OnlyForest_from2007=pd.merge(joined_obsplot_forest, speciestable, on='speciesKey', how='left')
SpeciesTable_OnlyForest_from2007=SpeciesTable_OnlyForest_from2007[['species','kingdom','phylum','class','order','family','genus','specificEpithet',
                                                                   'vernacularName ']]
SpeciesTable_OnlyForest_from2007=SpeciesTable_OnlyForest_from2007.groupby('species').first().reset_index()

SpeciesTable_OnlyForest_from2007.to_excel(filepath+'SpeciesTableJustForestFrom2007.xlsx',encoding='latin-1',index=False)

Groupby : polygonID, number of observed species,maximum of organismquantity

In [104]:
agghow={'speciesKey':['count','first'], 'organismQuantity':['max']}
groupby=joined_obsplot_forest.sort_values('organismQuantity', ascending=False).groupby('polygonID').agg(agghow).reset_index()

groupby.to_csv(filepath+'forInfoBSc.csv',sep=";",index=False)

In [120]:
GroupbyForinfoBScTable=pd.read_csv(filepath+'forInfoBSc.csv',sep=';',encoding='latin-1')
GroupbyForinfoBScTable.dtypes

polygonID              int64
nofspecies             int64
speciesKey             int64
maxorganismQuantity    int64
dtype: object

In [122]:
ForInfoBSc_joinedcoordinates=pd.merge(GroupbyForinfoBScTable, plottable[['polygonID','footprintWKT']], on='polygonID', how='left')

ForInfoBSc_JoinedCoordinates_DomTax=pd.merge(ForInfoBSc_joinedcoordinates, speciestable, on='speciesKey', how='left')

ForInfoBSc_JoinedCoordinates_DomTax=ForInfoBSc_JoinedCoordinates_DomTax[(ForInfoBSc_JoinedCoordinates_DomTax["maxorganismQuantity"]>50)]

ForInfoBSc_JoinedCoordinates_DomTax.to_csv(filepath+'ForInfoBSc_JoinedCoordinates.csv',sep=";",index=False)